In [1]:
import sys
import torch
from torch.utils.data import DataLoader
from torch import nn
from transformers import BertTokenizer, BertTokenizerFast, BertForSequenceClassification, BertModel, BertConfig, ElectraModel, ElectraConfig, ElectraTokenizer, RobertaForSequenceClassification, RobertaModel, RobertaTokenizer
from tokenizers import BertWordPieceTokenizer


sys.path.insert(0, '../')
from models import load_model, VanillaKoBert, VanillaXLMSeqClf, VanillaXLMBase
from dataset import REDataset, load_data
from config import Config, ModelType, PreTrainedType, PreProcessType

In [9]:
data = REDataset(model_type=ModelType.XLMSequenceClf, preprocess_type=PreProcessType.EM, device='cpu')

Getting dataset for XLMSequenceClf...
Load raw data...	preprocessing for 'EntityMarker'...	done!
Load Tokenizer for EntityMarker...	done!
Apply Tokenization...	No specific preprocessing for XLMSequenceClf yet. Just return tokenized outputs.


In [10]:
loader = DataLoader(data, batch_size=4)

In [11]:
for sents, labels in loader:
    break

In [13]:
model.resize_token_embeddings(len(data.tokenizer))

In [18]:
model(**sents).logits.size()

torch.Size([4, 42])

In [7]:
model = VanillaXLMSeqClf(42)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [ ]:
model(**)

In [13]:
data = load_data(Config.Train)

In [16]:
tokenizer = RobertaTokenizer.from_pretrained(PreTrainedType.XLMRoberta)

In [18]:
tokenizer(data['relation_state'][0])

{'input_ids': [0, 43998, 711, 10172, 46873, 8906, 12410, 43998, 6800, 16948, 43998, 11936, 48, 46747, 49171, 43998, 15113, 15375, 45209, 16948, 711, 45209, 27969, 10674, 46747, 27969, 10470, 47649, 9357, 11582, 43998, 18537, 21402, 46747, 48, 21402, 47649, 13859, 18537, 45209, 18164, 11582, 47649, 13859, 7487, 46747, 17772, 16948, 45209, 9357, 27, 43998, 7487, 11423, 48280, 711, 47672, 18537, 14285, 45209, 17772, 48, 45209, 9085, 48, 45209, 5543, 48, 45209, 27969, 10674, 47672, 17772, 48, 45209, 9085, 48, 45209, 5543, 48, 45209, 14292, 11936, 1640, 26902, 18448, 43, 43998, 27, 7471, 46747, 6248, 7471, 47649, 10674, 11936, 1640, 35805, 2462, 43, 46873, 7487, 7471, 46747, 17772, 23133, 43998, 48, 4394, 45209, 15375, 7487, 6, 46747, 46, 20024, 47672, 18537, 14285, 45209, 17772, 48, 45209, 9085, 48, 45209, 9085, 10470, 48280, 7471, 46747, 17772, 16948, 45209, 9357, 27, 43998, 7487, 11423, 48280, 711, 46747, 7258, 5782, 45209, 8210, 711, 45209, 8210, 4394, 46747, 46, 4394, 43998, 2023, 4958

In [9]:
data = REDataset(model_type=ModelType.XLMSequenceClf, preprocess_type=PreProcessType.EM ,  device='cpu')

Load raw data...	preprocessing for 'EntityMarker'...	done!
Load Tokenizer for EntityMarker...	


done!
Update token_type_ids: 9000it [00:00, 31142.69it/s]


KeyError: 'token_type_ids'

In [6]:
model

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Drop

In [ ]:
XLMRobertaModel.from_pretrained()

In [2]:
data = REDataset(preprocess_type=PreProcessType.EM, device='cpu')

Load raw data...	preprocessing for 'EntityMarker'...	done!
Load Tokenizer for EntityMarker...	done!
Update token_type_ids: 9000it [00:00, 32704.66it/s]


In [3]:
loader = DataLoader(data, batch_size=4)